In [1]:
# Necessary imports
import tweepy
from tweepy import OAuthHandler
import json
import sqlite3
import re
import pandas as pd
import csv

In [2]:
cd

/Users/rorypulvino


In [3]:
# Getting tweet grab function
% run Dropbox/Python/jupyter-blog/content/tweet_dumper.py

In [4]:
get_all_tweets("rorypul", "M")

getting tweets before 539173446182526976
...10 tweets downloaded so far


In [5]:
df = pd.DataFrame()
df = df.append(pd.read_csv('rorypul_tweets.csv'))
df

id           created_at  \
rorypul M  730921414178439169  2016-05-13 00:43:39   
        M  726062391134281728  2016-04-29 14:55:38   
        M  639474464388988929  2015-09-03 16:26:07   
        M  628239016417185792  2015-08-03 16:20:28   
        M  571787355134291969  2015-02-28 21:41:43   
        M  571787241665781760  2015-02-28 21:41:16   
        M  571782596335677441  2015-02-28 21:22:48   
        M  539889222996348928  2014-12-02 21:09:56   
        M  539782469382574081  2014-12-02 14:05:44   
        M  539173446182526977  2014-11-30 21:45:41   

                                                        text  
rorypul M  RT @poverty_action: New study in Science: "Pos...  
        M  RT @landportal: Good Read: Reshaping the Debat...  
        M  Mobile technology for non-judicial grievance m...  
        M  The Land Battle: 15 Organizations Defending La...  
        M                                #removeterranceburk  
        M  RT @IEDPBrazil: When in Brazil, #GoBlue! Thank...  
        M  RT @IEDPBrazil: A gente acaba de chegar em Bra...  
        M  I just made a donation to U-M for #GivingBlued...  
        M  IPSA should receive an extra $1,000 today for ...  
        M  @IEDPBrazil http://t.co/5QZNk7ra65 Brazil's ne...

In [6]:
names = [('rorypul', 'M'), ('rorypul', 'F')]

In [ ]:
################# #####################
# Figure out time delay in get_all_tweets so as not to go over request limit
# Figure out how to send csv to sql table

In [7]:
df = pd.DataFrame()
for (name, sex) in names:
    get_all_tweets(name, sex)
    df = df.append(pd.read_csv('%s_tweets.csv' % name))

getting tweets before 539173446182526976
...10 tweets downloaded so far
getting tweets before 539173446182526976
...10 tweets downloaded so far


In [8]:
df

id           created_at  \
rorypul M  730921414178439169  2016-05-13 00:43:39   
        M  726062391134281728  2016-04-29 14:55:38   
        M  639474464388988929  2015-09-03 16:26:07   
        M  628239016417185792  2015-08-03 16:20:28   
        M  571787355134291969  2015-02-28 21:41:43   
        M  571787241665781760  2015-02-28 21:41:16   
        M  571782596335677441  2015-02-28 21:22:48   
        M  539889222996348928  2014-12-02 21:09:56   
        M  539782469382574081  2014-12-02 14:05:44   
        M  539173446182526977  2014-11-30 21:45:41   
        F  730921414178439169  2016-05-13 00:43:39   
        F  726062391134281728  2016-04-29 14:55:38   
        F  639474464388988929  2015-09-03 16:26:07   
        F  628239016417185792  2015-08-03 16:20:28   
        F  571787355134291969  2015-02-28 21:41:43   
        F  571787241665781760  2015-02-28 21:41:16   
        F  571782596335677441  2015-02-28 21:22:48   
        F  539889222996348928  2014-12-02 21:09:56   
        F  539782469382574081  2014-12-02 14:05:44   
        F  539173446182526977  2014-11-30 21:45:41   

                                                        text  
rorypul M  RT @poverty_action: New study in Science: "Pos...  
        M  RT @landportal: Good Read: Reshaping the Debat...  
        M  Mobile technology for non-judicial grievance m...  
        M  The Land Battle: 15 Organizations Defending La...  
        M                                #removeterranceburk  
        M  RT @IEDPBrazil: When in Brazil, #GoBlue! Thank...  
        M  RT @IEDPBrazil: A gente acaba de chegar em Bra...  
        M  I just made a donation to U-M for #GivingBlued...  
        M  IPSA should receive an extra $1,000 today for ...  
        M  @IEDPBrazil http://t.co/5QZNk7ra65 Brazil's ne...  
        F  RT @poverty_action: New study in Science: "Pos...  
        F  RT @landportal: Good Read: Reshaping the Debat...  
        F  Mobile technology for non-judicial grievance m...  
        F  The Land Battle: 15 Organizations Defending La...  
        F                                #removeterranceburk  
        F  RT @IEDPBrazil: When in Brazil, #GoBlue! Thank...  
        F  RT @IEDPBrazil: A gente acaba de chegar em Bra...  
        F  I just made a donation to U-M for #GivingBlued...  
        F  IPSA should receive an extra $1,000 today for ...  
        F  @IEDPBrazil http://t.co/5QZNk7ra65 Brazil's ne...

In [28]:
# Creating a sql table of twitter handles we are interested in
conn = sqlite3.connect('Dropbox/Python/blog/content/Twitter_soccer/twitter_sports.sqlite')
cur = conn.cursor()

cur.executescript("""
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Tweet_sports;

CREATE TABLE User (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    Name TEXT UNIQUE,
    Handle   TEXT UNIQUE,
    Sex TEXT
);

CREATE TABLE Tweet_sports (
    id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    user_id INTEGER, 
    Tweet TEXT
);

""")

In [29]:
cur.executescript('''
INSERT INTO User (Name, Handle, Sex) VALUES ("The Equalizer", "EqualizerSoccer", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Alexi Lalas", "AlexiLalas", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Eric Wynalda", "EricWynalda", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Taylor Twellman", "TaylorTwellman", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Kate Markgraf", "katemarkgraf", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Julie Foudy", "JulieFoudy", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Brad Friedel", "BradFriedel", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Julie Stewart-Binks", "JSB_FOX", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Adriana Monsalve", "AndrianaMonsalve", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Kyle Martino", "kylemartino", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Brandi Chastain", "brandichastain", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Aly Wagner", "alywagner", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Abby Wambach", "AbbyWambach", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Kate Abdo", "kate_abdo", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Gabriel Marcotti", "Marcotti", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Grant Wahl", "GrantWahl", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Maximilano Bretos", "mbretosESPN", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Antoinetta Collins", "AntoinettaESPN", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Rob Stone", "RobStoneONFOX", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Warren Barton", "warrenbarton2", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Arlo White", "arlowhite", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Alejandro Moreno", "AleMorenoESPN", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Robbie Mustoe", "robbiemustoe", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Steve Bower", "SteveBowercomm", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Robbie Earle & Robbie Mustoe", "The2RobbiesNBC", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Keith Costigan", "KeithCostigan", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Cat Whitehilll", "catwhitehill4", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Bob Ley", "BobLeyESPN", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Tony DiCicco", "tonysocc", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Mark Jackson", "MarkJackson13", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Hannah Storm", "HannahStormESPN", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Doris Burke", "DorisBurkeNBA", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Michael Smith", "michaelsmith", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Jemele Hill", "jemelehill", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Jalen Rose", "jalenrose", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Sage Steele", "sagesteele", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Michelle Beadle", "MichelleDBeadle", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Tracy McGrady", "Real_T_Mac", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("LaChina Robinson", "LaChinaRobinson", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Stephen Bardo", "stephenbardo", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Lindsay Czarniak", "lindsayczarniak", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Mark Jones", "MarkJonesESPN", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Holly Rowe", "sportsiren", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Heather Cox", "HeatherCoxNBC", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Marc Kestecher", "marckestecher", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Rebecca Lobo", "RebeccaLobo", "F");
INSERT INTO User (Name, Handle, Sex) VALUES ("Gene Wojciechowski", "GenoEspn", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Dave Pasch", "DavePasch", "M");
INSERT INTO User (Name, Handle, Sex) VALUES ("Scott van Pelt", "notthefakeSVP", "M");
''')

In [30]:
result = cur.execute('''SELECT Handle, Sex FROM User''')
a_list = result.fetchall()

In [31]:
a_list

[(u'EqualizerSoccer', u'F'),
 (u'AlexiLalas', u'M'),
 (u'EricWynalda', u'M'),
 (u'TaylorTwellman', u'M'),
 (u'katemarkgraf', u'F'),
 (u'JulieFoudy', u'F'),
 (u'BradFriedel', u'M'),
 (u'JSB_FOX', u'F'),
 (u'AndrianaMonsalve', u'F'),
 (u'kylemartino', u'M'),
 (u'brandichastain', u'F'),
 (u'alywagner', u'F'),
 (u'AbbyWambach', u'F'),
 (u'kate_abdo', u'F'),
 (u'Marcotti', u'M'),
 (u'GrantWahl', u'M'),
 (u'mbretosESPN', u'M'),
 (u'AntoinettaESPN', u'F'),
 (u'RobStoneONFOX', u'M'),
 (u'warrenbarton2', u'M'),
 (u'arlowhite', u'M'),
 (u'AleMorenoESPN', u'M'),
 (u'robbiemustoe', u'M'),
 (u'SteveBowercomm', u'M'),
 (u'The2RobbiesNBC', u'M'),
 (u'KeithCostigan', u'M'),
 (u'catwhitehill4', u'F'),
 (u'BobLeyESPN', u'M'),
 (u'tonysocc', u'M'),
 (u'MarkJackson13', u'M'),
 (u'HannahStormESPN', u'F'),
 (u'DorisBurkeNBA', u'F'),
 (u'michaelsmith', u'M'),
 (u'jemelehill', u'F'),
 (u'jalenrose', u'M'),
 (u'sagesteele', u'F'),
 (u'MichelleDBeadle', u'F'),
 (u'Real_T_Mac', u'M'),
 (u'LaChinaRobinson', u'F')

In [32]:
for i in range(0,len(a_list)):
    a_list[i] = str(a_list[i])
type(a_list[0])

str

In [49]:
names = []
for i in range(0,len(a_list)):
    handle = re.findall("\(u'(.*)', u'(.*)'\)", a_list[i])[0]
    #print handle
    names.append(handle)
names

[('EqualizerSoccer', 'F'),
 ('AlexiLalas', 'M'),
 ('EricWynalda', 'M'),
 ('TaylorTwellman', 'M'),
 ('katemarkgraf', 'F'),
 ('JulieFoudy', 'F'),
 ('BradFriedel', 'M'),
 ('JSB_FOX', 'F'),
 ('AndrianaMonsalve', 'F'),
 ('kylemartino', 'M'),
 ('brandichastain', 'F'),
 ('alywagner', 'F'),
 ('AbbyWambach', 'F'),
 ('kate_abdo', 'F'),
 ('Marcotti', 'M'),
 ('GrantWahl', 'M'),
 ('mbretosESPN', 'M'),
 ('AntoinettaESPN', 'F'),
 ('RobStoneONFOX', 'M'),
 ('warrenbarton2', 'M'),
 ('arlowhite', 'M'),
 ('AleMorenoESPN', 'M'),
 ('robbiemustoe', 'M'),
 ('SteveBowercomm', 'M'),
 ('The2RobbiesNBC', 'M'),
 ('KeithCostigan', 'M'),
 ('catwhitehill4', 'F'),
 ('BobLeyESPN', 'M'),
 ('tonysocc', 'M'),
 ('MarkJackson13', 'M'),
 ('HannahStormESPN', 'F'),
 ('DorisBurkeNBA', 'F'),
 ('michaelsmith', 'M'),
 ('jemelehill', 'F'),
 ('jalenrose', 'M'),
 ('sagesteele', 'F'),
 ('MichelleDBeadle', 'F'),
 ('Real_T_Mac', 'M'),
 ('LaChinaRobinson', 'F'),
 ('stephenbardo', 'M'),
 ('lindsayczarniak', 'F'),
 ('MarkJonesESPN', 'M'),


In [ ]:
names = []
for i in range(0,len(a_list)):
    handle = re.findall("u'(.*)',", a_list[i])
    names.append(handle)
names

In [ ]:
########################## Works to create df with columns for each dictionary key, but only gives last 20 tweets for some reason ###########
x = api.user_timeline('alexilalas')

df = pd.DataFrame()
for i in x:
   a = json.dumps(i._json)
   b = pd.read_json(a)
   df = df.append(b.ix[0] )

df

In [ ]:
######################## Need to check/run this and to build in a time delay!!! ######################
df = pd.DataFrame()
df['handle'] = []
for i in range(0,len(names)):
    data = api.user_timeline(names[i])
    
    for j in data:
        js = json.dumps(j._json)
        py_data = pd.read_json(js)
        df = df.append(py_data.ix[0])
        ############## Need to figure out how to plug in the handle into the 'Handle' column###########
    time.sleep(900) # delaying the loop 15 minutes to account for twitter regulations ######### check to make sure this is the right time delay

In [ ]:
###################### This works, but need to change to the loop below #####################
for status in tweepy.Cursor(api.user_timeline, id = "alexilalas").items(1):
    a = json.dumps(status._json)
    try: js = json.loads(str(a))
    except: js = None
    tweet = js["text"]
    cur.execute('''INSERT INTO `Tweet_sports` (Tweet) 
                    VALUES ( ? )''', ( tweet, ) )
    
################ NEED TO TEST THIS ##################
    # Grabs id numbers based on twitter handle
    handle = "alexilalas"
    cur.execute('SELECT id FROM User WHERE id = ? ', (handle, ))
    user_id = cur.fetchone()[0]
    
    # Links id number from User table to user_id in Tweet_sports table
    cur.execute('''INSERT INTO `Tweet_sports` (user_id) 
                    VALUES ( ? )''', ( user_id, ) )
    conn.commit()

In [38]:
######################## Need to check/run this and to build in a time delay!!! ######################
for i in range(0,len(names)):
    data = api.user_timeline(names[i]) # This could be problematic
    a = json.dumps(data._json)
    try: js = json.loads(str(a))
    except: js = None
    tweet = js["text"]
    cur.execute('''INSERT INTO `Tweet_sports` (Tweet) 
                    VALUES ( ? )''', ( tweet, ) )
    
    # Grabs id numbers based on twitter handle
    handle = names[i]
    cur.execute('SELECT id FROM User WHERE id = ? ', (handle, ))
    user_id = cur.fetchone()[0]
    
    # Links id number from User table to user_id in Tweet_sports table
    cur.execute('''INSERT INTO `Tweet_sports` (user_id) 
                    VALUES ( ? )''', ( user_id, ) )
    
    conn.commit()
